## Combining "Box Office Mojo" and "The Movie Database" data by movie title

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
## Importing both data sets

bom = pd.read_csv("./data/bom.movie_gross.csv.gz") ##  Box Office Mojo data
tmdb = pd.read_csv("./data/tmdb.movies.csv.gz") ## The Movie Database data
num = pd.read_csv("./data/tn.movie_budgets.csv.gz") ## The Numbers movie budget data

In [4]:
## Sampling each dataframe

bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [15]:
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [6]:
num.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [18]:
bom.columns
tmdb.columns
num.columns

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [19]:
## A list of unqiue movie titles from each list
bom['title'].nunique()  ## 3,386
tmdb['title'].nunique() ## 24,688
num['movie'].nunique()  ## 5,698

5698

In [52]:
## Converting TMDB data to datetime object
tmdb['date'] = pd.to_datetime(tmdb['release_date'])

tmdb['month'] = tmdb['date'].dt.month

tmdb['year'] = tmdb['date'].dt.year

## Converting NUM release_date to datetime object and adding month & year to new columns

num['date'] = pd.to_datetime(num['release_date'])

num['month'] = num['date'].dt.month

num['year'] = num['date'].dt.year

# slicing dataset 2010-2019
mask = (tmdb['year']<=2019) & (tmdb['year']>=2010)
tmdb_2 = tmdb.loc[mask].copy()

mask2 = (num['year']<=2019) & (num['year']>=2010)
num_2 = num.loc[mask2].copy()

In [23]:
tmdb_2.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,date,month,year
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,2010-11-19,11,2010
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,2010-03-26,3,2010
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,2010-05-07,5,2010
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,2010-07-16,7,2010
5,5,"[12, 14, 10751]",32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229,2010-02-11,2,2010


In [24]:
tmdb_2['title'].nunique() ## 24,501 --> Filtering out by year 2010-2019 didn't remove many entries

24501

Need to check for duplicates: 

In [53]:
num_2.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,date,month,year
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011-05-20,5,2011
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019-06-07,6,2019
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015-05-01,5,2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017-12-15,12,2017
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220",2015-12-18,12,2015


In [54]:
duplicates = num_2.loc[(num_2['movie'].duplicated(keep = False) == True)]
duplicates.sort_values("movie",axis = 0)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,date,month,year
38,39,"May 14, 2010",Robin Hood,"$210,000,000","$105,487,148","$322,459,006",2010-05-14,5,2010
408,9,"Nov 21, 2018",Robin Hood,"$99,000,000","$30,824,628","$84,747,441",2018-11-21,11,2018
3025,26,"Feb 22, 2013",Snitch,"$15,000,000","$42,930,462","$57,907,734",2013-02-22,2,2013
5351,52,"Dec 31, 2012",Snitch,"$850,000",$0,$0,2012-12-31,12,2012
5009,10,"Apr 9, 2010",The Square,"$1,900,000","$406,216","$740,932",2010-04-09,4,2010
5099,100,"Oct 25, 2013",The Square,"$1,500,000","$124,244","$176,262",2013-10-25,10,2013
2970,71,"Apr 5, 2013",Trance,"$16,000,000","$2,322,593","$22,594,052",2013-04-05,4,2013
5330,31,"Dec 31, 2012",Trance,"$950,000",$0,$0,2012-12-31,12,2012


In [106]:
## Merging Numbers-Budget data with Box Office Mojo data. 
## Perform 'Outer' join on movie title and year 

compare = pd.merge(num_2, bom, how = "outer", left_on = ['movie', 'year' ], right_on = ['title', 'year'])
compare.head(50)



,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,date,month,year,title,studio,domestic_gross_y,foreign_gross
0,2.0,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011-05-20,5.0,2011,Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,804600000
1,3.0,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019-06-07,6.0,2019,NaN,NaN,NaN,NaN
2,4.0,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015-05-01,5.0,2015,Avengers: Age of Ultron,BV,459000000.0,946400000
3,5.0,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017-12-15,12.0,2017,NaN,NaN,NaN,NaN
4,6.0,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220",2015-12-18,12.0,2015,NaN,NaN,NaN,NaN
5,7.0,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",2018-04-27,4.0,2018,Avengers: Infinity War,BV,678800000.0,"1,369.5"
6,9.0,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209",2017-11-17,11.0,2017,Justice League,WB,229000000.0,428900000
7,10.0,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923",2015-11-06,11.0,2015,Spectre,Sony,200100000.0,680600000
8,11.0,"Jul 20, 2012",The Dark Knight Rises,"$275,000,000","$448,139,099","$1,084,439,099",2012-07-20,7.0,2012,The Dark Knight Rises,WB,448100000.0,636800000
9,12.0,"May 25, 2018",Solo: A Star Wars Story,"$275,000,000","$213,767,512","$393,151,347",2018-05-25,5.0,2018,Solo: A Star Wars Story,BV,213800000.0,179200000


In [65]:
## Checking work: The duplicated values are the Null values from where the data sets did not overlap
## effictively concatenating the tables and merging any overlaps. 
compare.title.duplicated().sum()  #2175 - movie 

976

In [73]:
## Part II to the proof that we've got unique movies on each row
compare_duplicates = compare.loc[(compare['title'].duplicated(keep = False) == True)]
compare_duplicates.title.value_counts(dropna = False)

NaN          976
Bluebeard      2
Name: title, dtype: int64

In [98]:
## Now need to merge Movie and Title columns
## going to try fillna

#compare['movie_title'] = compare.movie.fillna(compare.title)

compare.loc[(compare['movie_title'].duplicated(keep = False) == True)].sort_values("movie_title", axis = 0)


,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,date,month,year,title,studio,domestic_gross_y,foreign_gross,movie title,movie_title
3663,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2016,A Monster Calls,Focus,3700000.0,43600000,A Monster Calls,A Monster Calls
538,41.0,"Jan 6, 2017",A Monster Calls,"$43,000,000","$3,740,823","$46,414,530",2017-01-06,1.0,2017,NaN,NaN,NaN,NaN,A Monster Calls,A Monster Calls
1452,67.0,"Dec 21, 2011",Albert Nobbs,"$7,500,000","$3,014,696","$8,539,003",2011-12-21,12.0,2011,NaN,NaN,NaN,NaN,Albert Nobbs,Albert Nobbs
2655,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2012,Albert Nobbs,RAtt.,3000000.0,2600000,Albert Nobbs,Albert Nobbs
2541,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2011,Beautiful Boy,Anch.,77200.0,NaN,Beautiful Boy,Beautiful Boy
876,95.0,"Oct 12, 2018",Beautiful Boy,"$25,000,000","$7,634,767","$13,031,424",2018-10-12,10.0,2018,NaN,NaN,NaN,NaN,Beautiful Boy,Beautiful Boy
2889,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2013,Believe,ORF,6200000.0,NaN,Believe,Believe
1699,13.0,"Dec 2, 2016",Believe,"$3,500,000","$890,303","$890,303",2016-12-02,12.0,2016,NaN,NaN,NaN,NaN,Believe,Believe
832,52.0,"Mar 21, 2014",Blood Ties,"$26,000,000","$42,472","$2,923,959",2014-03-21,3.0,2014,NaN,NaN,NaN,NaN,Blood Ties,Blood Ties
2572,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2011,Blood Ties,ALP,2800.0,NaN,Blood Ties,Blood Ties


In [103]:

## Checking to see if the merge attempt worked
compare.loc[:,['movie','title','movie_title']].describe()

,movie,title,movie_title
count,2191,3387,4363
unique,2187,3386,4341
top,The Square,Bluebeard,The Square
freq,2,2,3
